In [39]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import numpy as np

In [40]:
mnist_data = loadmat('mnist_49_3000.mat')
X = torch.tensor(mnist_data['x'], dtype=torch.float64)
y = torch.tensor(mnist_data['y'][0], dtype=torch.float64)
X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=1000, random_state=42)

batch_size = 1

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [41]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(X.shape[1], 1).float()
        
    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        return out

In [42]:
learning_rate = 0.01
regularization_parameter = 0.01

model = LogisticRegression()
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [45]:
num_epochs = 100
for epoch in range(num_epochs):
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()
        inputs = inputs.float()
        outputs = model(inputs.T)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (784x1 and 3000x1)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        predicted = (outputs >= 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')